## Datathon Pós-Tech Data Analytics FIAP
# Fase 5 

## Diego Peres Hatanaka

In [28]:
# Bibliotecas

import json, re, unicodedata
import pandas as pd
from pathlib import Path
import numpy as np


In [29]:
base = Path("C:/Users/dphat/OneDrive/Documentos/Cursos/FIAP/PosTech_DataAnalytics/fase5/Datathon Decision")
raw = base / "data" / "raw"
processed = base / "data" / "processed"

In [30]:
# Leitura dos arquivos json

# vagas.json
with open(raw/'vagas.json', 'r', encoding='utf-8') as vagas:
    arq_vagas = json.load(vagas)

# applicants.json
with open(raw/'applicants.json', 'r', encoding='utf-8') as applicants:
    arq_applicants = json.load(applicants)

# prospects.json
with open(raw/'prospects.json', 'r', encoding='utf-8') as prospects:
    arq_prospects = json.load(prospects)


# Tratamento do arquivo vagas.json

In [32]:
# -----------------------------
# Funções auxiliares
# -----------------------------
def slugify(s: str) -> str:
    """Remove acentos, espaços e símbolos p/ virar nome de coluna seguro."""
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    s = re.sub(r"[^A-Za-z0-9]+", "_", s)
    return s.strip("_").lower()

def to_bool_sim_nao(x):
    """Converte 'Sim'/'Não' (case-insensitive) em True/False."""
    if isinstance(x, str):
        lx = x.strip().lower()
        if lx in ("sim", "yes", "true"):
            return True
        if lx in ("nao", "não", "no", "false"):
            return False
    return x  # deixa como está se não for Sim/Não

# -----------------------------
# Flatten (uma linha por vaga)
# -----------------------------
rows = []
for job_id, payload in arq_vagas.items():
    # achata níveis (informacoes_basicas, perfil_vaga, beneficios, etc.)
    flat = pd.json_normalize(payload, sep="__")
    flat.insert(0, "job_id", str(job_id))
    rows.append(flat)

df_vagas = pd.concat(rows, ignore_index=True)

# -----------------------------
# Limpeza de nomes de colunas
# -----------------------------
# 1) padronização das chaves portuguesas → ascii, snake_case
df_vagas.columns = [slugify(c) for c in df_vagas.columns]

# 2) renomeia alguns campos para nomes mais amigáveis
ren = {
    # Informações Básicas:
    "informacoes_basicas_data_requicisao": "data_requisicao",
    "informacoes_basicas_limite_esperado_para_contratacao": "limite_contratacao",
    "informacoes_basicas_titulo_vaga": "titulo_vaga",
    "informacoes_basicas_vaga_sap": "vaga_sap",
    "informacoes_basicas_cliente": "cliente",
    "informacoes_basicas_solicitante_cliente": "solicitante_cliente",
    "informacoes_basicas_empresa_divisao": "empresa_divisao",
    "informacoes_basicas_requisitante": "requisitante",
    "informacoes_basicas_analista_responsavel": "analista_responsavel",
    "informacoes_basicas_tipo_contratacao": "tipo_contratacao",
    "informacoes_basicas_prazo_contratacao": "prazo_contratacao",
    "informacoes_basicas_objetivo_vaga": "objetivo_vaga",
    "informacoes_basicas_prioridade_vaga": "prioridade_vaga",
    "informacoes_basicas_origem_vaga": "origem_vaga",
    "informacoes_basicas_superior_imediato": "superior_imediato",
    "informacoes_basicas_nome": "nome",
    "informacoes_basicas_telefone": "telefone",

    # Perfil da Vaga:
    "perfil_vaga_pais": "vaga_pais",
    "perfil_vaga_estado": "vaga_estado",
    "perfil_vaga_cidade": "vaga_cidade",
    "perfil_vaga_bairro": "vaga_bairro",
    "perfil_vaga_regiao": "vaga_regiao",
    "perfil_vaga_local_trabalho": "local_trabalho",
    "perfil_vaga_vaga_especifica_para_pcd": "vaga_pcd",
    "perfil_vaga_faixa_etaria": "faixa_etaria",
    "perfil_vaga_horario_trabalho": "horario_trabalho",
    "perfil_vaga_nivel_profissional": "nivel_profissional",
    "perfil_vaga_nivel_academico": "nivel_academico",
    "perfil_vaga_nivel_ingles": "nivel_ingles",
    "perfil_vaga_nivel_espanhol": "nivel_espanhol",
    "perfil_vaga_outro_idioma": "outro_idioma",
    "perfil_vaga_areas_atuacao": "area_atuacao",
    "perfil_vaga_principais_atividades": "principais_atividades",
    "perfil_vaga_competencia_tecnicas_e_comportamentais": "compet_tecnicas_comport",
    "perfil_vaga_demais_observacoes": "demais_obs",
    "perfil_vaga_viagens_requeridas": "viagens_requeridas",
    "perfil_vaga_equipamentos_necessarios": "equipamentos_necessarios",
    "beneficios_valor_venda": "valor_renda",
    "beneficios_valor_compra_1": "valor_compra1",
    "beneficios_valor_compra_2": "valor_compra2",
    "informacoes_basicas_data_inicial": "data_incial",
    "informacoes_basicas_data_final": "data_final",
    "perfil_vaga_habilidades_comportamentais_necessarias": "habilidades_comportamentais",
    "informacoes_basicas_nome_substituto": "nome_substituto",
}
df_vagas = df_vagas.rename(columns={k: v for k, v in ren.items() if k in df_vagas.columns})

# -----------------------------
# Conversões úteis
# -----------------------------
# 1) Datas (tenta converter formatos tipo dd-mm-aaaa; ignora '00-00-0000')
for col in ["data_requisicao", "limite_contratacao"]:
    if col in df_vagas.columns:
        df_vagas[col] = df_vagas[col].replace({"00-00-0000": None, "": None})
        df_vagas[col] = pd.to_datetime(df_vagas[col], dayfirst=True, errors="coerce")

# 2) Campos Sim/Não → booleanos
bool_like_cols = [
    "vaga_sap",
    "vaga_pcd",
]
for col in bool_like_cols:
    if col in df_vagas.columns:
        df_vagas[col] = df_vagas[col].map(to_bool_sim_nao)

# -----------------------------
# Resultado
# -----------------------------
print(df_vagas.shape)
display(df_vagas.head(3))
display(df_vagas.info())


# 1) CSV completo
out_csv = processed / "vagas_vf.csv"
df_vagas.to_csv(out_csv, index=False, encoding="utf-8-sig") 

(14081, 45)


,job_id,data_requisicao,limite_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,...,demais_obs,viagens_requeridas,equipamentos_necessarios,valor_renda,valor_compra1,valor_compra2,data_incial,data_final,habilidades_comportamentais,nome_substituto
0,5185,2021-05-04,NaT,Operation Lead -,False,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,2021-05-04,NaT,Consultor PP/QM Sênior,False,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,2021-05-04,NaT,ANALISTA PL/JR C/ SQL,False,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   job_id                       14081 non-null  object        
 1   data_requisicao              14081 non-null  datetime64[ns]
 2   limite_contratacao           7989 non-null   datetime64[ns]
 3   titulo_vaga                  14081 non-null  object        
 4   vaga_sap                     14081 non-null  bool          
 5   cliente                      14081 non-null  object        
 6   solicitante_cliente          14081 non-null  object        
 7   empresa_divisao              14081 non-null  object        
 8   requisitante                 14081 non-null  object        
 9   analista_responsavel         14081 non-null  object        
 10  tipo_contratacao             14081 non-null  object        
 11  prazo_contratacao            14081 non-nu

None

# Tratamento do arquivo applicant.json

In [33]:
# -----------------------------
# Funções auxiliares
# -----------------------------
def slugify(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    s = re.sub(r"[^A-Za-z0-9]+", "_", s)
    return s.strip("_").lower()

def to_bool_sim_nao(x):
    if isinstance(x, str):
        lx = x.strip().lower()
        if lx in ("sim", "yes", "true"): return True
        if lx in ("nao", "não", "no", "false"): return False
    return x

# -----------------------------
# Flatten (uma linha por candidato)
# -----------------------------
rows = []
for app_id, payload in arq_applicants.items():        # <<-- usa a chave do dict como applicant_id
    flat = pd.json_normalize(payload, sep="_")
    flat.insert(0, "applicant_id", str(app_id))       # <<-- garante a coluna applicant_id
    rows.append(flat)

df_applicants = pd.concat(rows, ignore_index=True)

# -----------------------------
# Padroniza nomes de colunas
# -----------------------------
df_applicants.columns = [slugify(c) for c in df_applicants.columns]

# Se existir algum código interno, usa-o quando disponível para sobrescrever o applicant_id
for alt in [
    "infos_basicas_codigo_profissional", "informacoes_pessoais_codigo_profissional",
    "codigo_profissional", "codigo", "id_candidato", "candidato_id", "codigo_candidato"
]:
    if alt in df_applicants.columns:
        df_applicants["applicant_id_alt"] = df_applicants[alt].astype(str)
        # Mantém o applicant_id do topo se não estiver vazio; senão usa o alternativo
        df_applicants["applicant_id"] = df_applicants["applicant_id"].where(
            df_applicants["applicant_id"].notna() & (df_applicants["applicant_id"].astype(str) != ""),
            df_applicants["applicant_id_alt"]
        )
        df_applicants.drop(columns=["applicant_id_alt"], inplace=True)
        break

df_applicants["applicant_id"] = df_applicants["applicant_id"].astype(str)

# -----------------------------
# Renomeia alguns campos para nomes mais amigáveis
# (as chaves abaixo já estão no formato slugify)
# -----------------------------
ren = {
    "infos_basicas_telefone_recado": "telefone_recado",
    "infos_basicas_telefone": "telefone",
    "infos_basicas_objetivo_profissional": "objetivo_profissional",
    "infos_basicas_data_criacao": "data_criacao",
    "infos_basicas_inserido_por": "inserido_por",
    "infos_basicas_email": "email",
    "infos_basicas_local": "local",
    "infos_basicas_sabendo_de_nos_por": "sabendo_de_nos_por",
    "infos_basicas_data_atualizacao": "data_atualizacao",
    "infos_basicas_codigo_profissional": "codigo_profissional",
    "infos_basicas_nome": "nome",
    "informacoes_pessoais_data_aceite": "data_aceite",
    "informacoes_pessoais_nome": "nome2",
    "informacoes_pessoais_cpf": "cpf",
    "informacoes_pessoais_fonte_indicacao": "fonte_indicacao",
    "informacoes_pessoais_email": "email2",
    "informacoes_pessoais_email_secundario": "email_secundario",
    "informacoes_pessoais_data_nascimento": "data_nascimento",
    "informacoes_pessoais_telefone_celular": "telefone_celular",
    "informacoes_pessoais_telefone_recado": "telefone_recado2",
    "informacoes_pessoais_sexo": "sexo",
    "informacoes_pessoais_estado_civil": "estado_civil",
    "informacoes_pessoais_pcd": "pcd",
    "informacoes_pessoais_endereco": "endereco",
    "informacoes_pessoais_skype": "skype",
    "informacoes_pessoais_url_linkedin": "url_linkedin",
    "informacoes_pessoais_facebook": "facebook",
    "informacoes_profissionais_titulo_profissional": "titulo_profissional",
    "informacoes_profissionais_area_atuacao": "area_atuacao",
    "informacoes_profissionais_conhecimentos_tecnicos": "conhecimentos_tecnicos",
    "informacoes_profissionais_certificacoes": "certificacoes",
    "informacoes_profissionais_outras_certificacoes": "outras_certificacoes",
    "informacoes_profissionais_remuneracao": "remuneracao",
    "informacoes_profissionais_nivel_profissional": "nivel_profissional",
    "formacao_e_idiomas_nivel_academico": "nivel_academico",
    "formacao_e_idiomas_nivel_ingles": "nivel_ingles",
    "formacao_e_idiomas_nivel_espanhol": "nivel_espanhol",
    "formacao_e_idiomas_outro_idioma": "outro_idioma",
    "formacao_e_idiomas_instituicao_ensino_superior": "inst_ensino_superior",
    "formacao_e_idiomas_cursos": "cursos",
    "formacao_e_idiomas_ano_conclusao": "ano_conclusao",
    "informacoes_profissionais_qualificacoes": "qualificacoes",
    "informacoes_profissionais_experiencias": "experiencias",
    "formacao_e_idiomas_outro_curso": "outro_curso",
    "cargo_atual_id_ibrati": "id_ibrati",
    "cargo_atual_email_corporativo": "email_corporativo",
    "cargo_atual_cargo_atual": "cargo_atual",
    "cargo_atual_projeto_atual": "projeto_atual",
    "cargo_atual_cliente": "cliente",
    "cargo_atual_unidade": "unidade",
    "cargo_atual_data_admissao": "data_admissao",
    "cargo_atual_data_ultima_promocao": "data_ultima_promocao",
    "cargo_atual_nome_superior_imediato": "nome_superior_imediato",
    "cargo_atual_email_superior_imediato": "email_superior_imediato",
}
df_applicants = df_applicants.rename(columns={k: v for k, v in ren.items() if k in df_applicants.columns})

# -----------------------------
# Conversões úteis
# -----------------------------
# Datas dd-mm-aaaa (corrige checagem: no DF APPLICANTS)
date_cols = ["data_criacao","data_atualizacao","data_nascimento","data_aceite","data_admissao","data_ultima_promocao"]
for col in date_cols:
    if col in df_applicants.columns:
        df_applicants[col] = df_applicants[col].replace({"00-00-0000": None, "": None})
        df_applicants[col] = pd.to_datetime(df_applicants[col], dayfirst=True, errors="coerce")

# Booleans (Sim/Não) típicos em APPLICANTS
for col in ["pcd"]:
    if col in df_applicants.columns:
        df_applicants[col] = df_applicants[col].map(to_bool_sim_nao)

# Remove duplicatas por applicant_id (se houver)
df_applicants = df_applicants.drop_duplicates(subset=["applicant_id"])

# -----------------------------
# Resultado
# -----------------------------
print(df_applicants.shape)
display(df_applicants.head(10))
df_applicants.info()

# 1) CSV completo
out_csv = processed / "applicants_vf.csv"
df_applicants.to_csv(out_csv, index=False, encoding="utf-8-sig") 


C:\Users\dphat\AppData\Local\Temp\ipykernel_43376\2784574907.py:119: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_applicants[col] = pd.to_datetime(df_applicants[col], dayfirst=True, errors="coerce")
C:\Users\dphat\AppData\Local\Temp\ipykernel_43376\2784574907.py:119: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_applicants[col] = pd.to_datetime(df_applicants[col], dayfirst=True, errors="coerce")


(42482, 58)


,applicant_id,cv_pt,cv_en,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,...,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,assistente administrativo\n\n\nsantosbatista\n...,,,(11) 97048-2708,,2021-11-10 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,31001,formação acadêmica\nensino médio (2º grau) em ...,,,(11) 93723-4396,Analista Administrativo,2021-11-10 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,31002,objetivo: área administrativa | financeira\n\n...,,,(11) 92399-9824,Administrativo | Financeiro,2021-11-10 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,31003,formação\nensino médio completo\ninformática i...,,,(11) 98100-1727,Área administrativa,2021-11-10 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
4,31004,última atualização em 09/11/2021\n­ sp\n\nensi...,,,(11) 92517-2678,,2021-11-10 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
5,31005,"brasileira\n41 anos, casada\nobjetivo: adminis...",,,(11) 94989-4617,Administrativa/logística e ou comercial,2021-11-10 09:19:56,Clara Rios,maya_fonseca@gmail.com,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
6,31006,objetivo\nárea financeira / controladoria / fa...,,,(11) 94846-8897,Analista Administrativo Financeiro,2021-11-10 09:22:20,Clara Rios,thomas_rodrigues@gmail.com,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
7,31007,belo horizonte/mg – cep: 30.626-710\nhabilitaç...,,,(31) 99869-5928,Desejo fazer parte do time da empresa oferecen...,2021-11-10 09:27:18,Clara Rios,pedro_miguel_nascimento@hotmail.com,"Belo Horizonte, Minas Gerais",...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
8,31008,última atualização em 04/11/2021\n\nresumo\nex...,,,(11) 92883-2196,,2021-11-10 09:32:58,Maria Clara Pires,ágatha_pereira@gmail.com,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
9,31009,"nasceu em 28 de maio de 1990, 31 anos.\nfemini...",,,(11) 95355-4464,,2021-11-10 09:39:27,Maria Clara Pires,paulo_novaes@gmail.com,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 58 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   applicant_id                      42482 non-null  object        
 1   cv_pt                             42482 non-null  object        
 2   cv_en                             42482 non-null  object        
 3   telefone_recado                   42482 non-null  object        
 4   telefone                          42482 non-null  object        
 5   objetivo_profissional             42482 non-null  object        
 6   data_criacao                      42463 non-null  datetime64[ns]
 7   inserido_por                      42482 non-null  object        
 8   email                             42482 non-null  object        
 9   local                             42482 non-null  object        
 10  sabendo_de_nos_por                42482 non-nu

# Tratamento do arquivo prospects.json

In [35]:
# -----------------------------
# Funções auxiliares
# -----------------------------
def slugify(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    s = re.sub(r"[^A-Za-z0-9]+", "_", s)
    return s.strip("_").lower()

# -----------------------------
# Prospects → 1 linha por candidato por vaga
# -----------------------------
rows = []

for job_id, payload in arq_prospects.items():
    # lista de candidatos dentro da vaga
    lst = (
        payload.get("prospects")
        or payload.get("prospections")
        or payload.get("prospeccoes")
        or payload.get("lista")
        or []
    )
    if not isinstance(lst, list) or len(lst) == 0:
        continue

    # normaliza a lista para linhas
    tmp = pd.json_normalize(lst)

    # metadados da vaga
    tmp.insert(0, "job_id", str(job_id))
    tmp["titulo"] = payload.get("titulo")
    tmp["modalidade"] = payload.get("modalidade")

    rows.append(tmp)

# se não houver linhas, cria DF vazio com colunas padrão
if rows:
    df_prospects = pd.concat(rows, ignore_index=True)
else:
    df_prospects = pd.DataFrame(columns=[
        "job_id", "codigo", "nome", "situacao_candidado",
        "data_candidatura", "ultima_atualizacao", "comentario",
        "recrutador", "titulo", "modalidade"
    ])

# -----------------------------
# Renomeia campos para padrão
# -----------------------------
ren = {
    "codigo": "applicant_id",
    "nome": "prospect_name",
    "situacao_candidado": "prospect_status",  # mantém o nome como veio no JSON de origem
    "comentario": "prospect_comment",
    "recrutador": "recruiter",
    "data_candidatura": "candidatura_dt",
    "ultima_atualizacao": "atualizacao_dt",
    "titulo": "job_title",
    "modalidade": "job_modalidade",
}
df_prospects = df_prospects.rename(columns={k: v for k, v in ren.items() if k in df_prospects.columns})

# -----------------------------
# Tipos e normalizações úteis
# -----------------------------
# IDs como string
for c in ("job_id", "applicant_id"):
    if c in df_prospects.columns:
        df_prospects[c] = df_prospects[c].astype(str)

# Datas dd-mm-aaaa → datetime (ignora '00-00-0000' e vazios)
for col in ("candidatura_dt", "atualizacao_dt"):
    if col in df_prospects.columns:
        df_prospects[col] = df_prospects[col].replace({"00-00-0000": None, "": None})
        df_prospects[col] = pd.to_datetime(df_prospects[col], dayfirst=True, errors="coerce")

# Status normalizado (útil p/ rotulagem futura)
if "prospect_status" in df_prospects.columns:
    df_prospects["prospect_status_norm"] = (
        df_prospects["prospect_status"]
        .astype(str).str.strip().str.lower()
    )

# Comentário: comprimento (proxy de engajamento)
if "prospect_comment" in df_prospects.columns:
    df_prospects["prospect_comment_len"] = df_prospects["prospect_comment"].apply(
        lambda x: len(x) if isinstance(x, str) else 0
    )

# -----------------------------
# Ordena colunas (primeiro as principais)
# -----------------------------
ordem = [
    "job_id", "applicant_id", "prospect_name", "prospect_status", "prospect_status_norm",
    "candidatura_dt", "atualizacao_dt", "prospect_comment", "prospect_comment_len",
    "recruiter", "job_title", "job_modalidade"
]
cols = [c for c in ordem if c in df_prospects.columns] + [c for c in df_prospects.columns if c not in ordem]
df_prospects = df_prospects[cols]

# -----------------------------
# Resultado
# -----------------------------
print(df_prospects.shape)
display(df_prospects.head(10))
df_prospects.info()

# 1) CSV completo
out_csv = processed / "prospects_vf.csv"
df_prospects.to_csv(out_csv, index=False, encoding="utf-8-sig")


(53759, 12)


,job_id,applicant_id,prospect_name,prospect_status,prospect_status_norm,candidatura_dt,atualizacao_dt,prospect_comment,prospect_comment_len,recruiter,job_title,job_modalidade
0,4530,25632,José Vieira,Encaminhado ao Requisitante,encaminhado ao requisitante,2021-03-25,2021-03-25,"Encaminhado para - PJ R$ 72,00/hora",36,Ana Lívia Moreira,CONSULTOR CONTROL M,
1,4530,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,encaminhado ao requisitante,2021-03-22,2021-03-23,"encaminhado para - R$ 6.000,00 – CLT Full , n...",67,Ana Lívia Moreira,CONSULTOR CONTROL M,
2,4531,25364,Sra. Yasmin Fernandes,Contratado pela Decision,contratado pela decision,2021-03-17,2021-04-12,Data de Inicio: 12/04/2021,26,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,
3,4531,25360,Alexia Barbosa,Encaminhado ao Requisitante,encaminhado ao requisitante,2021-03-17,2021-03-17,,0,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,
4,4533,26338,Arthur Almeida,Contratado pela Decision,contratado pela decision,2021-04-29,2021-05-18,,0,Stella Vieira,2021-2605708-Microfocus Application Life Cycle...,
5,4533,24645,Dante Sampaio,Desistiu,desistiu,2021-04-27,2021-04-27,Profissional desistiu da vaga. Motivo : Na ver...,121,Yasmin da Rosa,2021-2605708-Microfocus Application Life Cycle...,
6,4534,26361,Ana Luiza Vieira,Documentação PJ,documentação pj,2021-04-28,2021-05-11,Aguardando confirmação de inicio _,35,Manuella Carvalho,2021-2605711-Microfocus QTP - UFT Automation T...,
7,4534,26205,Isabella da Cruz,Desistiu,desistiu,2021-04-23,2021-04-30,"profissional não atende ou responde ligações, ...",64,Ana Camargo,2021-2605711-Microfocus QTP - UFT Automation T...,
8,4534,26003,Maria Helena Peixoto,Não Aprovado pelo Cliente,não aprovado pelo cliente,2021-04-08,2021-04-16,"""Conversando com a candidata, foi exposto que ...",254,Carolina Araújo,2021-2605711-Microfocus QTP - UFT Automation T...,
9,4534,25509,Dra. Gabrielly Cassiano,Desistiu,desistiu,2021-03-22,2021-03-22,No momento não avalia,21,Dra. Luara Siqueira,2021-2605711-Microfocus QTP - UFT Automation T...,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   job_id                53759 non-null  object        
 1   applicant_id          53759 non-null  object        
 2   prospect_name         53759 non-null  object        
 3   prospect_status       53759 non-null  object        
 4   prospect_status_norm  53759 non-null  object        
 5   candidatura_dt        53759 non-null  datetime64[ns]
 6   atualizacao_dt        49846 non-null  datetime64[ns]
 7   prospect_comment      53759 non-null  object        
 8   prospect_comment_len  53759 non-null  int64         
 9   recruiter             53759 non-null  object        
 10  job_title             53759 non-null  object        
 11  job_modalidade        53759 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 4.9+ MB


In [21]:
# Garantir tipos consistentes nas chaves
for c in ("job_id",):
    if c in df_vagas.columns:     df_vagas[c] = df_vagas[c].astype(str)
    if c in df_prospects.columns: df_prospects[c] = df_prospects[c].astype(str)
for c in ("applicant_id",):
    if c in df_prospects.columns:  df_prospects[c] = df_prospects[c].astype(str)
    if c in df_applicants.columns: df_applicants[c] = df_applicants[c].astype(str)


In [26]:
# =========================
# Merge completo (3 bases)
# =========================

# Chaves padrão
KEY_JOB = "job_id"
KEY_APP = "applicant_id"

def _prefix_except_keys(df: pd.DataFrame, prefix: str, keys: list[str]) -> pd.DataFrame:
    """Prefixa todas as colunas exceto as chaves informadas."""
    ren = {c: f"{prefix}{c}" for c in df.columns if c not in keys}
    out = df.rename(columns=ren).copy()
    # padroniza dtype das chaves para garantir match no merge
    for k in keys:
        if k in out.columns:
            out[k] = out[k].astype("string")
    return out

# 1) Prefixos para evitar colisão (mantendo as chaves sem prefixo)
pros_pref = _prefix_except_keys(df_prospects,  "prospect__", [KEY_JOB, KEY_APP])
vagas_pref = _prefix_except_keys(df_vagas,     "job__",      [KEY_JOB])
apps_pref  = _prefix_except_keys(df_applicants,"app__",      [KEY_APP])

# 2) Merges (PROSPECTS -> VAGAS -> APPLICANTS)
def _safe_merge(left, right, on, how="left", validate="m:1"):
    """Tenta validar cardinalidade; se falhar, faz merge sem validate e avisa."""
    try:
        return left.merge(right, on=on, how=how, validate=validate)
    except Exception as e:
        print(f"⚠️ Aviso: merge({on}) sem validação devido a: {e}")
        return left.merge(right, on=on, how=how)

tmp = _safe_merge(pros_pref, vagas_pref, on=KEY_JOB, validate="m:1")  # vários prospects p/ 1 vaga
df_consolidado = _safe_merge(tmp, apps_pref, on=KEY_APP, validate="m:1")  # vários prospects p/ 1 candidato

# 3) ID único da relação vaga×candidato
df_consolidado["pair_id"] = (
    df_consolidado[KEY_JOB].astype("string") + "::" + df_consolidado[KEY_APP].astype("string")
)

# 4) (Opcional) organizar colunas: chaves e pair_id primeiro
front = ["pair_id", KEY_JOB, KEY_APP]
rest = [c for c in df_consolidado.columns if c not in front]
df_consolidado = df_consolidado[front + rest]

print(df_consolidado.shape)
display(df_consolidado.head(3))


(53759, 114)


,pair_id,job_id,applicant_id,prospect__prospect_name,prospect__prospect_status,prospect__prospect_status_norm,prospect__candidatura_dt,prospect__atualizacao_dt,prospect__prospect_comment,prospect__prospect_comment_len,...,app__id_ibrati,app__email_corporativo,app__cargo_atual,app__projeto_atual,app__cliente,app__unidade,app__data_admissao,app__data_ultima_promocao,app__nome_superior_imediato,app__email_superior_imediato
0,4530::25632,4530,25632,José Vieira,Encaminhado ao Requisitante,encaminhado ao requisitante,2021-03-25,2021-03-25,"Encaminhado para - PJ R$ 72,00/hora",36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,4530::25529,4530,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,encaminhado ao requisitante,2021-03-22,2021-03-23,"encaminhado para - R$ 6.000,00 – CLT Full , n...",67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,4531::25364,4531,25364,Sra. Yasmin Fernandes,Contratado pela Decision,contratado pela decision,2021-03-17,2021-04-12,Data de Inicio: 12/04/2021,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [27]:
# Salvando arquivo em disco


# -----------------------------
# 1) Funções auxiliares
# -----------------------------
def _to_bool_sim_nao(x):
    if x is None or (isinstance(x, float) and pd.isna(x)) or x == '':
        return pd.NA
    s = str(x).strip().lower()
    if s in {'sim','yes','true','1','y'}:  return True
    if s in {'nao','não','no','false','0','n'}: return False
    return pd.NA  # qualquer outra coisa vira NA

def _ensure_scalar(x):
    # se houver listas/dicts/tuplas em alguma coluna object, serializa pra JSON
    if isinstance(x, (list, dict, tuple, set)):
        try:
            return json.dumps(x, ensure_ascii=False)
        except Exception:
            return str(x)
    return x


# ===== 2) Normalizar colunas booleanas (Sim/Não → True/False/NA) =====
# detecta prováveis colunas booleanas (ajuste o filtro se quiser)
bool_cols = [c for c in df_consolidado.columns
             if any(k in c.lower() for k in ['pcd','sap','flag','boolean'])]

def to_boolean_series(s: pd.Series) -> pd.Series:
    # garante tipo string para trabalhar com .str
    s = s.astype("string")
    # normaliza
    s_norm = s.str.strip().str.lower()
    # trata vazios e representações de nulos
    s_norm = s_norm.replace({"": pd.NA, "nan": pd.NA, "<na>": pd.NA})
    # mapeia valores
    true_vals  = {"sim","yes","true","1","y"}
    false_vals = {"nao","não","no","false","0","n"}
    out = pd.Series(pd.NA, index=s_norm.index, dtype="object")
    out[s_norm.isin(true_vals)]  = True
    out[s_norm.isin(false_vals)] = False
    return out.astype("boolean")

for col in bool_cols:
    df_consolidado[col] = to_boolean_series(df_consolidado[col])

# garantia extra para a coluna citada (se não entrou no filtro por algum motivo)
if "job__vaga_pcd" in df_consolidado.columns:
    df_consolidado["job__vaga_pcd"] = to_boolean_series(df_consolidado["job__vaga_pcd"])

# ===== 3) Converter objetos não escalares para string (se houver) =====
# (mantém o seu comportamento anterior, mas sem mexer nos booleanos já arrumados)
import json
obj_cols = df_consolidado.select_dtypes(include=["object"]).columns
for col in obj_cols:
    df_consolidado[col] = df_consolidado[col].apply(
        lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (list, dict, tuple, set)) else x
    )
    # strings vazias → NA (opcional)
    df_consolidado[col] = df_consolidado[col].replace("", pd.NA)

# ===== 4) Salvar Parquet =====
output_file = processed / "decision_consolidated.parquet"
df_consolidado.to_parquet(output_file, index=False)
print(f"✅ Arquivo salvo em: {output_file} | Linhas: {len(df_consolidado)}")


✅ Arquivo salvo em: C:\Users\dphat\OneDrive\Documentos\Cursos\FIAP\PosTech_DataAnalytics\fase5\Datathon Decision\data\processed\decision_consolidated.parquet | Linhas: 53759
